In [1]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from scipy import ndimage
from skimage import filters, morphology, io
import shutil
import warnings

In [2]:
# Define the mapping of specific values to RGB colors
value_to_rgb = {
    1: [255, 48, 48],      # fire brick1
    2: [64, 224, 208],     # turquoise
    3: [72, 118, 255],     # royalblue1
    4: [255, 255, 0],      # yellow
    5: [255, 255, 255],    # white
    6: (192, 192, 192),    # silver
    7: [255, 0, 255],      # magenta
}

In [3]:
file_path = '/data/nas/Cth/MRI_segmentation/'
file_list = os.listdir(file_path)

size_path = '/data/nas/Cth/size_total'
size_list = os.listdir(size_path)

print(size_list)

['si_(339, 480)', 'si_(335, 480)', 'si_(325, 480)', 'si_(360, 528)', 'si_(345, 512)', 'si_(360, 480)', 'si_(355, 480)', 'si_(360, 512)', 'si_(315, 480)', 'si_(345, 480)', 'si_(365, 512)', 'si_(380, 480)', 'si_(365, 528)', 'si_(300, 480)', 'si_(317, 512)', 'si_(310, 480)', 'si_(360, 640)', 'si_(340, 480)', 'si_(365, 480)', 'si_(385, 480)', 'si_(330, 480)', 'si_(320, 480)']


In [4]:
# Loading real path
def tar_space(png):
    file_path = '/data/nas/Cth/MRI_segmentation/'
    file_path_01 = os.path.join(file_path,png,f"{png}_nuc_t1_resample_axial2d")
    file_path_02 = os.path.join(file_path,png,"GT_rCSF_apLVc5_hippCSF")
    # print(file_path_01,file_path_02)
    return file_path_01,file_path_02

In [5]:
# Control ostu area
def create_brain_mask(image, adjustment_factor=0.2):
    threshold = filters.threshold_otsu(image)
    adjusted_threshold = threshold * adjustment_factor
    binary_mask = image > adjusted_threshold
    labeled_array, num_features = ndimage.label(binary_mask)
    sizes = ndimage.sum(binary_mask, labeled_array, range(num_features + 1))
    mask = sizes == sizes.max()
    brain_mask = mask[labeled_array]
    convex_hull = morphology.convex_hull_image(brain_mask)

    # Make rect mask
    rows = np.any(convex_hull, axis=1)
    cols = np.any(convex_hull, axis=0)
    min_row, max_row = np.where(rows)[0][[0, -1]]
    min_col, max_col = np.where(cols)[0][[0, -1]]
    rectangular_mask = np.zeros_like(image, dtype=bool)
    rectangular_mask[min_row:max_row+1, min_col:max_col+1] = True
    
    return rectangular_mask
    

In [6]:
# Crop mask
def crop_mask(image, mask):
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)
    min_row, max_row = np.where(rows)[0][[0, -1]]
    min_col, max_col = np.where(cols)[0][[0, -1]]
    cropped_image = image[min_row:max_row+1, min_col:max_col+1]
    
    return cropped_image

In [7]:
# Make new .nii file
def make_nii(or_nii_data, new_nii_data, save_path, file_name):
    output_nii_file = os.path.join(save_path, f"{file_name}.nii")
    extracted_nii = nib.Nifti1Image(new_nii_data, or_nii_data.affine)
    nib.save(extracted_nii, output_nii_file)

In [8]:
# Padding
def pading(crop):
    sq_size = 0
    if crop.shape[0] > crop.shape[1]:
        sq_size = crop.shape[0]
        pad_r = int((sq_size - crop.shape[0]) / 2)
        pad_c = int((sq_size - crop.shape[1]) / 2)
        if ((sq_size - crop.shape[1]) % 2) == 1:
            pad = np.pad(crop, ((pad_r, pad_r), (pad_c, pad_c + 1)), 'constant', constant_values=0)
        else:
            pad = np.pad(crop, ((pad_r, pad_r), (pad_c, pad_c)), 'constant', constant_values=0)

    else:
        sq_size = crop.shape[1]
        pad_r = int((sq_size - crop.shape[0]) / 2)
        pad_c = int((sq_size - crop.shape[1]) / 2)

        if ((sq_size - crop.shape[1]) % 2) == 1:
            pad = np.pad(crop, ((pad_r, pad_r + 1), (pad_c, pad_c)), 'constant', constant_values=0)
        else:
            pad = np.pad(crop, ((pad_r, pad_r), (pad_c, pad_c)), 'constant', constant_values=0)

    return pad

In [9]:
# Move GT
def move_GT(fp2, fp2_files, fp4_work_n, fp4_work_p):
    for nii in fp2_files:
        if nii.endswith('.nii'):
            nii_path_2 = os.path.join(fp2, nii)
            shutil.copy(nii_path_2, os.path.join(fp4_work_n, nii))  

In [10]:
# Map values to RGB colors
def map_values_to_rgb(img, value_to_rgb):
    # Create an empty array for the RGB image
    rgb_img = np.zeros((*img.shape, 3), dtype=np.uint8)

    for value, color in value_to_rgb.items():
        mask = (img == value)
        rgb_img[mask] = color

    return rgb_img

In [11]:
# Make new .png file - Colorful
def make_png(new_nii_data, save_path, file_name):
    # Map values to RGB
    rgb_img = map_values_to_rgb(new_nii_data, value_to_rgb)
    
    # Save the RGB image as a png file
    output_png_file = os.path.join(save_path, f"{file_name}.png")
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        io.imsave(output_png_file, rgb_img)

In [12]:
# Make new .png file - gray
def save_png(new_nii_data, save_path, file_name):
    fig, ax = plt.subplots(figsize=(new_nii_data.shape[1] / 100, new_nii_data.shape[0] / 100), dpi=100)
    ax.imshow(new_nii_data, cmap='gray')
    ax.axis('off')
    output_png_file = os.path.join(save_path, f"{file_name}.png")
    plt.savefig(output_png_file, bbox_inches='tight', pad_inches=0)
    plt.close(fig)

In [15]:
# Main code

# Decide the work place
for png_size in size_list:  # for png_size in size_list: ; Run for all size.
    print(png_size)
    png_path = os.path.join(size_path, png_size)
    png_list = os.listdir(png_path)

    for png_name in png_list:  # for png_name in png_list: ; Run on all files of that size.
        print(png_name)
        png = png_name.split('.')[0]  # working file name

        work_space = os.path.join('/data/nas/Cth/image_space/', png_size, png)
        # os.makedirs(work_space,exist_ok=True)
        
        # PNG path
        fp1_work_p = os.path.join(work_space, 'PNG', f'{png}_SQ_PNG')  # saving loc SQ
        fp2_work_p = os.path.join(work_space, 'PNG', f'{png}_PAD_PNG')  # saving loc PAD
        fp3_work_p = os.path.join(work_space, 'PNG', f'{png}_OR_PNG')  # Real original image
        fp4_work_p = os.path.join(work_space, 'PNG', f'{png}_GTOR_PNG')  # GT original image
        fp5_work_p = os.path.join(work_space, 'PNG', f'{png}_GTSQ_PNG')  # saving loc GTSQ
        fp6_work_p = os.path.join(work_space, 'PNG', f'{png}_GTPAD_PNG')  # saving loc GT PAD
        # os.makedirs(fp1_work_p, exist_ok=True)
        os.makedirs(fp2_work_p, exist_ok=True)
        # os.makedirs(fp3_work_p, exist_ok=True)
        # os.makedirs(fp4_work_p, exist_ok=True)
        # os.makedirs(fp5_work_p, exist_ok=True)
        os.makedirs(fp6_work_p, exist_ok=True)

        # NII path
        fp1_work_n = os.path.join(work_space, 'NII', f'{png}_SQ_NII')  # saving loc SQ
        fp2_work_n = os.path.join(work_space, 'NII', f'{png}_PAD_NII')  # saving loc PAD
        fp3_work_n = os.path.join(work_space, 'NII', f'{png}_OR_NII')  # Real original nii
        fp4_work_n = os.path.join(work_space, 'NII', f'{png}_GTOR_NII')  # GT original nii
        fp5_work_n = os.path.join(work_space, 'NII', f'{png}_GTSQ_NII')  # saving loc GTSQ
        fp6_work_n = os.path.join(work_space, 'NII', f'{png}_GTPAD_NII')  # saving loc GT PAD
        # os.makedirs(fp1_work_n, exist_ok=True)
        os.makedirs(fp2_work_n, exist_ok=True)
        # os.makedirs(fp3_work_n, exist_ok=True)
        os.makedirs(fp4_work_n, exist_ok=True)
        # os.makedirs(fp5_work_n, exist_ok=True)
        os.makedirs(fp6_work_n, exist_ok=True)

        fp1, fp2 = tar_space(png)
        fp1_files = os.listdir(fp1)
        fp2_files = os.listdir(fp2)
    
        move_GT(fp2, fp2_files, fp4_work_n, fp4_work_p)

        fp4_files = os.listdir(fp4_work_n)
        
        # Find biggest mask
        mask = None
        mask_area = 0
        for nii in fp1_files:
            # Load Original.nii
            nii_path_1 = os.path.join(fp1, nii)
            nii_data_1 = nib.load(nii_path_1)
            nii_data_1 = nii_data_1.get_fdata()
            nii_data_1 = np.rot90(nii_data_1)
            nii_name = nii.split('.')[0]
            
            # Making & Choosing mask
            mask1 = create_brain_mask(nii_data_1)
            area = np.sum(mask1)
            if area > mask_area:
                mask = mask1
                mask_area = area
        
        # GT
        for GT in fp4_files:
            nii_path_2 = os.path.join(fp4_work_n, GT)
            nii_data_2 = nib.load(nii_path_2)
            nii_data_2_nii = nii_data_2.get_fdata()
            nii_data_2_np = np.rot90(nii_data_2_nii)
            nii_name_2 = GT.split('.')[0]

            # Original file
            # make_png(nii_data_2_np, fp4_work_p, nii_name_2)

            # Crop mask area
            crop = crop_mask(nii_data_2_np, mask)
            # make_png(crop, fp5_work_p, nii_name_2)
            # make_nii(nii_data_2, np.rot90(crop,3), fp5_work_n, nii_name_2)

            # Padding area
            pad = pading(crop)
            make_png(pad, fp6_work_p, nii_name_2)
            make_nii(nii_data_2, np.rot90(pad,3), fp6_work_n, nii_name_2)

        # Real
        for nii in fp1_files:
            # Load Original.nii
            nii_path_1 = os.path.join(fp1, nii)
            nii_data_1 = nib.load(nii_path_1)
            nii_data_1_nii = nii_data_1.get_fdata()
            nii_data_1_np = np.rot90(nii_data_1_nii)
            nii_name = nii.split('.')[0]

            # Original file
            # save_png(nii_data_1_np, fp3_work_p, nii_name)
            # make_nii(nii_data_1, np.rot90(nii_data_1_np,3), fp3_work_n, nii_name)

            # Crop mask area
            crop = crop_mask(nii_data_1_np, mask)
            # save_png(crop, fp1_work_p, nii_name)
            # make_nii(nii_data_1, np.rot90(crop,3), fp1_work_n, nii_name)

            # Padding area
            pad = pading(crop)
            save_png(pad, fp2_work_p, nii_name)
            make_nii(nii_data_1, np.rot90(pad,3), fp2_work_n, nii_name)

si_(339, 480)
IDEA_235_160108.png
si_(335, 480)
IDEA_1307_190204.png
si_(325, 480)
NRCDB_581_121121.png
NRCDB_052_130125.png
IDEA_052_120429.png
NRCDB_466_120917.png
si_(360, 528)
NRCDB_656_150915.png
si_(345, 512)
IDEA_1417_190510.png
si_(360, 480)
NRCDB_1185_160812.png
IDEA_198_141214.png
IDEA_746_180509.png
IDEA_644_180311.png
IDEA_1270_190509.png
AM_SM_074_170627.png
SMC_THK_067_160923.png
IDEA_532_170930.png
IDEA_1357_190529.png
IDEA_539_170708.png
IDEA_526_170826.png
NRCDB_446_160529.png
NRCDB_533_151112.png
IDEA_517_180124.png
NRCDB_782_160807.png
IDEA_483_170626.png
NRCDB_163_190913.png
IDEA_1129_190317.png
IDEA_931_190127.png
NRCDB_434_170807.png
NRCDB_137_140102.png
IDEA_614_180106.png
IDEA_791_180310.png
NRCDB_1238_161016.png
IDEA_1030_180524.png
IDEA_448_170624.png
NRCDB_020_150904.png
NRCDB_483_160420.png
IDEA_1391_190819.png
NRCDB_788_160617.png
AM_SM_114_161218.png
NRCDB_415_160503.png
NRCDB_727_170111.png
IDEA_501_170410.png
IDEA_231_160911.png
NRCDB_716_160212.png
IDEA

In [37]:
import shutil

fin_path = '/data/nas/Cth/image_space'
fin_list = os.listdir(fin_path)
#fin_list

for size in fin_list:
    path = os.path.join(fin_path, size)
    f_list = os.listdir(path)
    
    for name in f_list:
        end_path = os.path.join(fin_path, size,name,'NII',f'{name}_GTOR_NII')
        print(end_path)
        if os.path.isdir(end_path):
            shutil.rmtree(end_path)

/data/nas/Cth/image_space/si_(339, 480)/IDEA_235_160108/NII/IDEA_235_160108_GTOR_NII
/data/nas/Cth/image_space/si_(335, 480)/IDEA_1307_190204/NII/IDEA_1307_190204_GTOR_NII
/data/nas/Cth/image_space/si_(325, 480)/NRCDB_581_121121/NII/NRCDB_581_121121_GTOR_NII
/data/nas/Cth/image_space/si_(325, 480)/NRCDB_052_130125/NII/NRCDB_052_130125_GTOR_NII
/data/nas/Cth/image_space/si_(325, 480)/NRCDB_466_120917/NII/NRCDB_466_120917_GTOR_NII
/data/nas/Cth/image_space/si_(325, 480)/IDEA_052_120429/NII/IDEA_052_120429_GTOR_NII
/data/nas/Cth/image_space/si_(360, 528)/NRCDB_656_150915/NII/NRCDB_656_150915_GTOR_NII
/data/nas/Cth/image_space/si_(345, 512)/IDEA_1417_190510/NII/IDEA_1417_190510_GTOR_NII
/data/nas/Cth/image_space/si_(360, 480)/IDEA_1206_190524/NII/IDEA_1206_190524_GTOR_NII
/data/nas/Cth/image_space/si_(360, 480)/NRCDB_1015_171009/NII/NRCDB_1015_171009_GTOR_NII
/data/nas/Cth/image_space/si_(360, 480)/NRCDB_1149_180107/NII/NRCDB_1149_180107_GTOR_NII
/data/nas/Cth/image_space/si_(360, 480)/NRC